# Ver maestro_picks (ordenado por Hora descendente)

In [ ]:
import pandas as pd
from pathlib import Path

def resolve_maestro() -> Path | None:
    cwd = Path.cwd()
    for p in [cwd] + list(cwd.parents):
        cand = p / "/home/pablo/Documentos/footystats/df_maestro/maestro_picks.pkl"
        if cand.exists():
            return cand
    return None

MAESTRO_PATH = resolve_maestro()
if MAESTRO_PATH is None:
    raise FileNotFoundError(f"No se encontró maestro_picks.pkl desde {Path.cwd()} hacia arriba")

df = pd.read_pickle(MAESTRO_PATH)
if "Estado" in df.columns:
    df["Estado"] = df["Estado"].astype(str).str.upper()

df.head(), MAESTRO_PATH

(  Corners_total Cuota_Justa     Estado Estado_EXE Fecha_ejecucion GPT_valido  \
 0           NaN         NaN  PENDIENTE        NaN      2026-01-03       True   
 1             9         NaN       ROJO        NaN      2026-01-01       True   
 2             8         NaN      VERDE        NaN      2026-01-01       True   
 3            10         NaN      VERDE        NaN      2026-01-01       True   
 4             9         NaN       ROJO        NaN      2026-01-01       True   
 
   Goles_OTeam              Hora ID_partido                    Liga  ...  \
 0         NaN  2026-01-03 05:30        NaN           Italy Serie A  ...   
 1         NaN  2026-01-01 09:00        NaN  England EFL League Two  ...   
 2         NaN  2026-01-01 09:00        NaN    England Championship  ...   
 3         NaN  2026-01-01 11:30        NaN    England Championship  ...   
 4         NaN  2026-01-01 09:00        NaN  England EFL League Two  ...   
 
   awayGoals_today awayID away_id competition_id homeG

In [ ]:
# Quitar duplicados para la vista (Hora, Match_id, Mercado)
dedup_keys = ["Hora", "Match_id", "Mercado"]
dedup_keys = [k for k in dedup_keys if k in df.columns]

df_view = df.copy()
dedup_subset = dedup_keys.copy()
if "Hora" in dedup_keys:
    df_view["_Hora_norm"] = pd.to_datetime(df_view["Hora"], errors="coerce")
    dedup_subset = ["_Hora_norm" if k == "Hora" else k for k in dedup_subset]
if "Match_id" in dedup_keys:
    df_view["_Match_id_norm"] = df_view["Match_id"].astype("string").str.strip()
    dedup_subset = ["_Match_id_norm" if k == "Match_id" else k for k in dedup_subset]
if "Mercado" in dedup_keys:
    df_view["_Mercado_norm"] = df_view["Mercado"].astype("string").str.strip().str.upper()
    dedup_subset = ["_Mercado_norm" if k == "Mercado" else k for k in dedup_subset]

if dedup_subset:
    dup_keep_last = df_view.duplicated(subset=dedup_subset, keep="last")
    df_view = df_view.loc[~dup_keep_last].copy()
    print(f"Filas tras dedup ({dedup_keys}): {len(df)} -> {len(df_view)}")

# Ordenar por Hora descendente y mostrar columnas clave
cols_to_show = ["Hora", "Liga", "Local", "Visitante", "Mercado", "Estado", "ODDS", "Match_id"]
cols_to_show = [c for c in cols_to_show if c in df_view.columns]

if "Hora" in df_view.columns:
    df_sorted = df_view.copy()
    df_sorted["Hora_ts"] = pd.to_datetime(df_sorted["Hora"], errors="coerce")
    df_sorted = df_sorted.sort_values("Hora_ts", ascending=False)
    display(df_sorted[cols_to_show].head(20))
else:
    print("No existe columna 'Hora' en el maestro.")


,Hora,Liga,Local,Visitante,Mercado,Estado,ODDS,Match_id
374,2026-01-08 14:15,Portugal LigaPro,Lusitania Lourosa,Leixões,OVER 1.5,PENDIENTE,1.32,8193882
372,2026-01-08 14:15,Portugal LigaPro,Lusitania Lourosa,NaN,BTTS YES,PENDIENTE,1.98,8193882
370,2026-01-08 14:00,England Premier League,Arsenal,Liverpool,GOL HT,PENDIENTE,1.32,8223558
373,2026-01-08 14:00,England Premier League,Arsenal,Liverpool,OVER 1.5,PENDIENTE,1.22,8223558
371,2026-01-08 14:00,England Premier League,Arsenal,NaN,BTTS YES,PENDIENTE,1.85,8223558
...,...,...,...,...,...,...,...,...
320,2025-12-28 05:00,Portugal LigaPro,Academico Viseu,NaN,BTTS YES,VERDE,1.78,8193835
317,2025-12-27 09:00,England Premier League,Arsenal,Brighton & Hove Albion,GOL HT,VERDE,1.33,8223657
319,2025-12-27 08:50,Saudi Arabia Professional League,Al Nassr,Al Akhdoud,Team Over 1.5,VERDE,NaN,8197934
318,2025-12-27 08:00,Italy Serie B,Empoli,Frosinone,OVER 1.5,VERDE,1.28,8173349
